In [1]:
import bt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from bt.core import Algo

In [2]:
data = pd.read_csv('./sp500_adj_close.csv',header=0, skiprows=1)
data = data[1:]
data.columns = ['datetime'] + data.columns[1:].to_list()
data['datetime'] = pd.to_datetime(data['datetime'])
data = data.set_index('datetime')
# special
data.drop(pd.to_datetime('2017-09-04'),inplace=True)
# data.loc['2018-01-01':]
data.head()

,A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,...,XLNX,XOM,XRAY,XRX,XYL,YUM,ZBH,ZBRA,ZION,ZTS
datetime,,,,,,,,,,,,,,,,,,,,,
2010-01-04,20.545469,4.496876,39.103363,6.583586,NaN,22.512976,8.74,20.074522,33.718582,37.090000,...,19.830650,46.349365,32.941868,14.571863,NaN,20.328438,55.501415,28.670000,11.726357,NaN
2010-01-05,20.322289,5.005957,38.870968,6.594968,NaN,22.352354,8.53,19.912334,33.926979,37.700001,...,19.580618,46.530319,32.550251,14.588748,NaN,20.258917,57.258369,28.620001,12.139812,NaN
2010-01-06,20.250088,4.798555,39.209885,6.490066,NaN,22.141001,8.40,20.022911,34.287651,37.619999,...,19.447794,46.932491,32.764709,14.453672,NaN,20.114088,57.239891,28.400000,13.195447,NaN
2010-01-07,20.223829,4.939965,39.200214,6.478067,NaN,21.785934,8.40,20.188793,34.255581,36.889999,...,19.252451,46.785034,33.193615,14.521203,NaN,20.108294,58.552982,27.690001,14.673337,NaN
2010-01-08,20.217272,4.845690,39.355148,6.521136,NaN,22.022644,8.23,20.292002,34.119335,36.689999,...,19.533739,46.597363,33.193615,14.470551,NaN,20.114088,57.323116,27.600000,14.435819,NaN


In [55]:
benchmark_data = pd.read_csv('./sp500_^SPGC_adj_close.csv',index_col=1)
benchmark_data = benchmark_data[['^GSPC']].reset_index('datatime').set_index('datatime').copy()
benchmark_data.index = pd.DatetimeIndex(benchmark_data.index)
benchmark_data



,^GSPC
datatime,
2010-01-04,1132.989990
2010-01-05,1136.520020
2010-01-06,1137.140015
2010-01-07,1141.689941
2010-01-08,1144.979980
...,...
2021-02-01,3773.860107
2021-02-02,3826.310059
2021-02-03,3830.169922


In [4]:
ema_50 = data.ewm(span=50, adjust=True).mean()
ema_100 = data.ewm(span=100, adjust=True).mean()
ema_150 = data.ewm(span=150, adjust=True).mean()
ema_200 = data.ewm(span=200, adjust=True).mean()

# 1. The current stock price is above both the 150-day and 200-day moving average
r_1 = (data > ema_150) + (data >ema_200)
# 2. The 150-day moving average is above the 200-day moving average
r_2 = ema_150 > ema_200
# 3. The 200-day moving average line is trending up for at least 1 month

# 4. The 50-day moving average is above both the 150-day and 200-day moving averages
r_4 = (ema_50 > ema_150) + (ema_50 > ema_200)
# 5. The current stock price is trading above the 50-day moving average
r_5 = data > ema_50
# 6. The current stock price is at least 30% above it’s 52-week low
low_52week = data.rolling(52 * 5).min()
r_6 = data >= low_52week * 1.3
# 7. The current stock price is within at least 25% of its 52-week high
high_52week = data.rolling(52 * 5).max()
r_7 = (data >= high_52week * 0.75) + (data <= high_52week * 1.25)
# 8. The relative strength ranking (as reported in Investor’s Business Daily) is no less than 70. Preferably 

signal = r_1 & r_2 & r_4 & r_5 & r_6 & r_7
signal

,A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,...,XLNX,XOM,XRAY,XRX,XYL,YUM,ZBH,ZBRA,ZION,ZTS
datetime,,,,,,,,,,,,,,,,,,,,,
2010-01-04,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2010-01-05,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2010-01-06,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2010-01-07,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2010-01-08,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-02-01,True,False,False,True,False,True,True,True,False,False,...,False,False,False,False,False,False,True,True,True,False
2021-02-02,True,False,False,True,False,True,True,True,False,True,...,False,False,True,False,False,True,True,True,True,False
2021-02-03,True,False,False,True,True,True,True,True,False,True,...,False,False,True,False,False,False,True,True,True,False


In [5]:
from pypfopt import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns


class WeighEqually_DIY(Algo):

    def __init__(self,optimial_type='min_volatility',risk_free_rate = 0.1):
        super(WeighEqually_DIY, self).__init__()
        self.optimial_type = optimial_type
        self.risk_free_rate = risk_free_rate

    def __call__(self, target):
        selected = target.temp['selected']
        n = len(selected)

        if n == 0:
            target.temp['weights'] = {}
        else:
            
            df = target.universe[selected]
            # Calculate expected returns and sample covariance
            mu = expected_returns.mean_historical_return(df)
            S = risk_models.sample_cov(df)
            
            # Optimise for maximal Sharpe ratio
            ef = EfficientFrontier(mu, S)
            
            if self.optimial_type == 'min_volatility':
                
                weights = ef.min_volatility()
                cleaned_weights = ef.clean_weights()
                target.temp['selected'] = list(cleaned_weights.keys())
                target.temp['weights'] = cleaned_weights
            elif self.optimial_type == 'max_sharpe':
                weights = ef.max_sharpe(risk_free_rate = self.risk_free_rate)

                # risk_free_rate (float, optional) – risk-free rate of borrowing/lending, defaults to 0.02. The period of the risk-free rate should correspond to the frequency of expected returns.

                cleaned_weights = ef.clean_weights()
                target.temp['selected'] = list(cleaned_weights.keys())
                target.temp['weights'] = cleaned_weights
        return True


In [6]:
# create the strategy #risk
s_mv_monthly = bt.Strategy('minimum volatility monthly', [bt.algos.RunMonthly(),
                       bt.algos.SelectWhere(signal),
                       WeighEqually_DIY(optimial_type='min_volatility'),
                       bt.algos.Rebalance()])
# create a backtest and run it
mv_monthly = bt.Backtest(s_mv_monthly, data,progress_bar=True)

# create the strategy. #return(max risk)
s_msr_monthly = bt.Strategy('maximum sharpe ratio monthly', [bt.algos.RunMonthly(),
                       bt.algos.SelectWhere(signal),
                       WeighEqually_DIY(optimial_type='max_sharpe',risk_free_rate = 0.05),
                       bt.algos.Rebalance()])
# create a backtest and run it
msr__monthly = bt.Backtest(s_msr_monthly, data,progress_bar=True)

In [7]:
# 'minimum volatility monthly'
# 'maximum sharpe ratio monthly'
# 'minimum volatility yearly'
# 'maximum sharpe ratio yearly'

In [8]:
# create the strategy #risk
s_mv_yearly = bt.Strategy('minimum volatility yearly', [bt.algos.RunYearly(),
                       bt.algos.SelectWhere(signal),
                       WeighEqually_DIY(optimial_type='min_volatility'),
                       bt.algos.Rebalance()])
# create a backtest and run it
mv_yearly = bt.Backtest(s_mv_yearly, data,progress_bar=True)

# create the strategy. #return(max risk)
s_msr_yearly = bt.Strategy('maximum sharpe ratio yearly', [bt.algos.RunYearly(),
                       bt.algos.SelectWhere(signal),
                       WeighEqually_DIY(optimial_type='max_sharpe',risk_free_rate = 0.1),
                       bt.algos.Rebalance()])
# create a backtest and run it
msr__yearly = bt.Backtest(s_msr_yearly, data,progress_bar=True)

In [9]:
benchmark_data

,^GSPC
datatime,
2010-01-04,1132.989990
2010-01-05,1136.520020
2010-01-06,1137.140015
2010-01-07,1141.689941
2010-01-08,1144.979980
...,...
2021-02-01,3773.860107
2021-02-02,3826.310059
2021-02-03,3830.169922


In [10]:
res = bt.run(mv_monthly, msr__monthly,mv_yearly,msr__yearly)  #,b_yearly)

minimum volatility monthly
0% [############################# ] 100% | ETA: 00:00:00maximum sharpe ratio monthly
0% [############################# ] 100% | ETA: 00:00:01minimum volatility yearly
0% [############################# ] 100% | ETA: 00:00:00maximum sharpe ratio yearly
0% [############################# ] 100% | ETA: 00:00:00

In [11]:
#AT the minimium risk the return is low and the maximum risk the return is high.

In [12]:
res.display()

Stat                 minimum volatility monthly    maximum sharpe ratio monthly    minimum volatility yearly    maximum sharpe ratio yearly
-------------------  ----------------------------  ------------------------------  ---------------------------  -----------------------------
Start                2010-01-03                    2010-01-03                      2010-01-03                   2010-01-03
End                  2021-02-05                    2021-02-05                      2021-02-05                   2021-02-05
Risk-free rate       0.00%                         0.00%                           0.00%                        0.00%

Total Return         221.98%                       434.28%                         157.90%                      470.29%
Daily Sharpe         0.84                          0.91                            0.67                         0.91
Daily Sortino        1.31                          1.43                            1.02                         1.40

In [13]:
res.display_monthly_returns()

  Year    Jan    Feb    Mar    Apr    May    Jun    Jul    Aug    Sep    Oct    Nov    Dec    YTD
------  -----  -----  -----  -----  -----  -----  -----  -----  -----  -----  -----  -----  -----
  2010   0      0      0      0      0      0      0      0      0      0      0      0      0
  2011   0      6.29   4.46   5.49   1.47  -1.06  -2.11  -1.92  -0.98   2.01   1.65   2.65  19
  2012  -1.43   1.33   2.48   3.15  -0.93   4.78   1.92  -3.56   1.63  -2.11  -1.04  -0.58   5.44
  2013   4.6    4.59   5.12   2.56  -3.34   0.68   6.42  -4.63   5.42   5.34  -0.16  -0.64  28.35
  2014  -0.45   3.3   -0.17   0.92   0.19   1.54  -5.21   3.57  -0.35   4.82   4.82  -0.13  13.16
  2015   1.74  -0.5   -0.84  -4.07   1.58   1.4    3.8   -3.15   0.48   2.91  -0.77   1.28   3.62
  2016  -0.1    1.66   5.03  -0.7    0.2    3.61   0.83  -0.9   -0.01  -2.68  -0.4   -1.83   4.55
  2017   0.62   3.09   0.56   2.54   3.32  -0.69   1.24   0.51   1.31   4.25   4.91  -0.84  22.7
  2018   4.96  -5.06   0.39

In [14]:
res.display_monthly_returns(1)

  Year    Jan    Feb     Mar    Apr    May    Jun    Jul    Aug    Sep    Oct    Nov     Dec    YTD
------  -----  -----  ------  -----  -----  -----  -----  -----  -----  -----  -----  ------  -----
  2010   0      0       0      0      0      0      0      0      0      0      0       0      0
  2011   0      0.15    4.46   3.36   3.38   0.54  -4.75  -0.96   1.23   3.49   1.08    1.76  14.26
  2012   2.93   8.1     4.93   2.78  -1.31   1.84  -1.21   1.01   1.14  -4.4    2.21    0.48  19.51
  2013   4.8    5.11    7.09   3.17   7.71  -4.24   6.36  -0.14   9.18   0.71   0.21    1.89  49.69
  2014  -1.03  11.84    0.55   2.39   3.34   7.65  -5.02   5.79  -6.29   4.19   6.91   -0.46  32.37
  2015  -1.41   4.76    0.9    0.92   1.42  -0.96   2.57  -3     -0.73   6.8    0.5    -3.23   8.38
  2016  -0.03  -1.66    3.21  -3.27   2.94   3.51   2.08   0.19   0.96   1.02  -1.16   -4.31   3.18
  2017   4.14   6.59   -1.19   0.92   7.32   0.81  -3.96   0.69   2.43   1.65   2.52    0.87  24.69
  2

In [15]:
res.display_monthly_returns(2)

  Year    Jan    Feb     Mar    Apr    May    Jun    Jul    Aug    Sep    Oct    Nov    Dec    YTD
------  -----  -----  ------  -----  -----  -----  -----  -----  -----  -----  -----  -----  -----
  2010   0      0       0      0      0      0      0      0      0      0      0      0      0
  2011   0      0       0      0      0      0      0      0      0      0      0      0      0
  2012  -0.44   2.06    2.52   3.25  -2.89   3.85   0.49  -1.6    0.63  -3.71   2.96  -1.57   5.33
  2013   3.76   4.56    6.18   0.26  -1.96  -0.8    5.85  -3.16   1      3.72   0.12   0.36  21.18
  2014  -0.4    7.33   -1.08  -1.91   1.56   0.8   -3.68   4.3   -1.29   5.59   4.67   1.9   18.61
  2015   1.76   1.05    0.08  -1.4    2.62  -2.36   6.37  -1.97  -0.05   2.56  -1.17   1.56   9.09
  2016   0.25  -1.02    4.94  -3.61  -2.32   1.71  -0.97  -1.15  -0.48  -1      1.19   2.38  -0.38
  2017  -0.39   2.02   -0.18   0.06   1.79  -1.22   3.27  -0.39   0.44   0.27   3.45   1.83  11.39
  2018   5.52  -

In [16]:
res.display_monthly_returns(3)

  Year    Jan    Feb     Mar    Apr    May    Jun    Jul    Aug    Sep     Oct    Nov    Dec    YTD
------  -----  -----  ------  -----  -----  -----  -----  -----  -----  ------  -----  -----  -----
  2010   0      0       0      0      0      0      0      0      0       0      0      0      0
  2011   0      0       0      0      0      0      0      0      0       0      0      0      0
  2012   4.34   9.01    6.59   2.09  -4.14   1.39  -4.15  -0.01   5.47  -12.75   4.2    2.46  13.26
  2013   3.72   2.99    6.72   4.2    8.73  -2.13   9.06  -3.64  11.03    0.16   2.12   1.46  53.07
  2014  -0.47  15.81   -0.4    2.52   3.83   8.39  -4.07   5.61  -8.49    0.22  -6.22   2.42  18.2
  2015  -0.87   6.5     0.93   2.26   1.04  -0.75   4.48  -1.18  -1.2     6.55  -1.83  -0.96  15.48
  2016   0.29  -3.42    8.45  -4.78   3.4    3.55   0.08  -1.71  -1.35   -2.39  -2.46   2.57   1.5
  2017   5.14   5.25    0.11   0.63   1.61  -1.1    4.37  -1.81   1.48    1.2    1.36   2.77  22.81
  2018  

In [110]:
res.prices[:]

,minimum volatility monthly,maximum sharpe ratio monthly,minimum volatility yearly,maximum sharpe ratio yearly
2010-01-03,100.000000,100.000000,100.000000,100.000000
2010-01-04,100.000000,100.000000,100.000000,100.000000
2010-01-05,100.000000,100.000000,100.000000,100.000000
2010-01-06,100.000000,100.000000,100.000000,100.000000
2010-01-07,100.000000,100.000000,100.000000,100.000000
...,...,...,...,...
2021-02-01,325.701902,525.390202,253.284725,553.271818
2021-02-02,322.997156,534.631524,255.370912,568.000873
2021-02-03,322.104430,526.465569,254.417208,559.559393
2021-02-04,318.601179,531.638869,255.674968,566.665152


In [17]:
res.plot().get_figure().savefig("output_res_plot.png")

In [18]:
res.to_csv(path='./dataset_res_performanceStats.csv')

In [19]:
res.prices[:].to_csv('./dataset_cumulative_returns_Income.csv')

In [20]:
cumulative_returns = res.prices[:]
cumulative_returns = cumulative_returns/100 -1
cumulative_returns

,minimum volatility monthly,maximum sharpe ratio monthly,minimum volatility yearly,maximum sharpe ratio yearly
2010-01-03,0.000000,0.000000,0.000000,0.000000
2010-01-04,0.000000,0.000000,0.000000,0.000000
2010-01-05,0.000000,0.000000,0.000000,0.000000
2010-01-06,0.000000,0.000000,0.000000,0.000000
2010-01-07,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...
2021-02-01,2.257019,4.253902,1.532847,4.532718
2021-02-02,2.229972,4.346315,1.553709,4.680009
2021-02-03,2.221044,4.264656,1.544172,4.595594
2021-02-04,2.186012,4.316389,1.556750,4.666652


In [21]:
cumulative_returns.to_csv('./dataset_cumulative_returns.csv')

In [22]:
cumulative_returns.plot(figsize=(16,9)).get_figure().savefig('output_cumulative_returns.png')

In [23]:
daily_return = res.prices[:].shift(1) - res.prices[:]
daily_return.tail()

,minimum volatility monthly,maximum sharpe ratio monthly,minimum volatility yearly,maximum sharpe ratio yearly
2021-02-01,-1.200305,-13.333770,-0.933249,-17.820067
2021-02-02,2.704746,-9.241321,-2.086188,-14.729055
2021-02-03,0.892726,8.165954,0.953704,8.441480
2021-02-04,3.503252,-5.173300,-1.257760,-7.105760
2021-02-05,-3.375063,-2.636447,-2.228082,-3.627870


In [24]:
daily_return.to_csv('./dataset_daily_returns.csv')

In [25]:
daily_return.plot(figsize=(16,9)).get_figure().savefig('output_daily_return.png')

In [26]:
benchmark_data.loc[:]

,^GSPC
datatime,
2010-01-04,1132.989990
2010-01-05,1136.520020
2010-01-06,1137.140015
2010-01-07,1141.689941
2010-01-08,1144.979980
...,...
2021-02-01,3773.860107
2021-02-02,3826.310059
2021-02-03,3830.169922


In [ ]:
sdasd

In [ ]:
###################################################################
# compare monthly stragies
###################################################################

In [94]:
benchmark_spical_data = benchmark_data.loc['2011-02-01':]
benchmark_daily_return = benchmark_spical_data.pct_change()

In [93]:
benchmark_data_cumulat_return = (1 + benchmark_daily_return).cumprod() - 1 
benchmark_data_cumulat_return.fillna(0,inplace=True)

In [103]:
cumulative_return_min_vol_monthly = cumulative_returns[['minimum volatility monthly']].loc['2011-02-01':]
cumulative_return_max_sr_monthly = cumulative_returns[['maximum sharpe ratio monthly']].loc['2011-02-01':]

,maximum sharpe ratio monthly
2011-02-01,0.000000
2011-02-02,-0.003012
2011-02-03,-0.003984
2011-02-04,0.013350
2011-02-07,0.009520
...,...
2021-02-01,4.253902
2021-02-02,4.346315
2021-02-03,4.264656
2021-02-04,4.316389


In [95]:
compare_min_v_monthly_vs_benchmark = pd.concat([benchmark_data_cumulat_return,min_vol_monthly_cumulative_return],axis = 1)
compare_max_sr_monthly_vs_benchmark = pd.concat([benchmark_data_cumulat_return,cumulative_return_max_sr_monthly],axis = 1)

In [96]:
compare_min_vol_monthly_vs_max_sr_monthly_vs_benchmark = pd.concat([benchmark_data_cumulat_return, min_vol_monthly_cumulative_return, cumulative_return_max_sr_monthly],axis = 1)

In [90]:
compare_min_v_monthly_vs_benchmark.to_csv('./dataset_compare_min_v_monthly_vs_benchmark.csv')
compare_max_sr_monthly_vs_benchmark.to_csv('./dataset_compare_max_sr_monthly_vs_benchmark.csv')
compare_min_vol_monthly_vs_max_sr_monthly_vs_benchmark.to_csv('./dataset_compare_min_v_monthly_vs_max_sr_monthly_vs_benchmark.csv')

In [89]:
compare_min_v_monthly_vs_benchmark.plot(figsize=(16,9)).get_figure().savefig('output_compare_min_v_monthly_vs_benchmark.png')
compare_max_sr_monthly_vs_benchmark.plot(figsize=(16,9)).get_figure().savefig('output_compare_max_sr_monthly_vs_benchmark.png')
compare_min_vol_monthly_vs_max_sr_monthly_vs_benchmark.plot(figsize=(16,9)).get_figure().savefig('output_compare_min_v_monthly_vs_max_sr_monthly_vs_benchmark.png')

In [ ]:
###################################################################
# compare yearly stragies
###################################################################

In [104]:
benchmark_spical_data_2012 = benchmark_data.loc['2012-01-01':]
benchmark_daily_return_2012 = benchmark_spical_data_2012.pct_change()
benchmark_daily_return_2012

,^GSPC
datatime,
2012-01-03,NaN
2012-01-04,0.000188
2012-01-05,0.002944
2012-01-06,-0.002537
2012-01-09,0.002262
...,...
2021-02-01,0.016052
2021-02-02,0.013898
2021-02-03,0.001009


In [105]:
benchmark_data_cumulat_return_2012 = (1 + benchmark_daily_return_2012).cumprod() - 1 
benchmark_data_cumulat_return_2012.fillna(0,inplace=True)

In [106]:
cumulative_return_min_vol_yearly = cumulative_returns[['minimum volatility yearly']].loc['2012-01-01':]
cumulative_return_max_sr_yearly = cumulative_returns[['maximum sharpe ratio yearly']].loc['2012-01-01':]

In [107]:
compare_min_v_yearly_vs_benchmark = pd.concat([benchmark_data_cumulat_return_2012,cumulative_return_min_vol_yearly],axis = 1)
compare_max_sr_yearly_vs_benchmark = pd.concat([benchmark_data_cumulat_return_2012,cumulative_return_max_sr_yearly],axis = 1)
compare_min_vol_yearly_vs_max_sr_yearly_vs_benchmark = pd.concat([benchmark_data_cumulat_return_2012, cumulative_return_min_vol_yearly, cumulative_return_max_sr_yearly],axis = 1)

In [108]:
compare_min_v_yearly_vs_benchmark.to_csv('./dataset_compare_min_v_yearly_vs_benchmark.csv')
compare_max_sr_yearly_vs_benchmark.to_csv('./dataset_compare_max_sr_yearly_vs_benchmark.csv')
compare_min_vol_yearly_vs_max_sr_yearly_vs_benchmark.to_csv('./dataset_compare_min_v_yearly_vs_max_sr_yearly_vs_benchmark.csv')

In [109]:
compare_min_v_yearly_vs_benchmark.plot(figsize=(16,9)).get_figure().savefig('output_compare_min_v_yearly_vs_benchmark.png')
compare_max_sr_yearly_vs_benchmark.plot(figsize=(16,9)).get_figure().savefig('output_compare_max_sr_yearly_vs_benchmark.png')
compare_min_vol_yearly_vs_max_sr_yearly_vs_benchmark.plot(figsize=(16,9)).get_figure().savefig('output_compare_min_v_yearly_vs_max_sr_yearly_vs_benchmark.png')

In [31]:
daily_return.mean()

minimum volatility monthly     -0.000787
maximum sharpe ratio monthly   -0.001541
minimum volatility yearly      -0.000554
maximum sharpe ratio yearly    -0.001658
benchmark                      -0.000856
dtype: float64

In [56]:
df[:].plot(figsize=(16,9)).get_figure().savefig('output_benchmark.png')

In [23]:
# Rolling Volatility
(res.prices.pct_change().rolling(window=12*20).std()*np.sqrt(252)).plot().get_figure().savefig("output_rolling_olatility.png")

[]

In [114]:
res.get_security_weights().plot().get_figure().savefig("./output_security_weights.png")

In [116]:
res.get_security_weights().to_csv('./dataset_security_weights_min_v_monthly.csv')
res.get_security_weights(1).to_csv('./dataset_security_weights_min_sr_monthly.csv')
res.get_security_weights(2).to_csv('./dataset_security_weights_min_v_yearly.csv')
res.get_security_weights(3).to_csv('./dataset_security_weights_min_sr_yearly.csv')

In [117]:
transactions_min_v_monthly = res.get_transactions('minimum volatility monthly')
transactions_max_sr_monthly = res.get_transactions('maximum sharpe ratio monthly')
transactions_min_v_yearly = res.get_transactions('minimum volatility yearly')
transactions_max_v_yearly = res.get_transactions('maximum sharpe ratio yearly')

In [118]:
transactions_min_v_monthly.to_csv('./dataset_transactions_min_v_monthly.csv')
transactions_max_sr_monthly.to_csv('./dataset_transactions_max_sr_monthly.csv')
transactions_min_v_yearly.to_csv('./dataset_transactions_min_v_yearly.csv')
transactions_max_v_yearly.to_csv('./dataset_transactions_max_r_yearly.csv')